In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Data Imputation

## Outline
* Review: How are data missing?
* Determining how data are missing
* Missing data imputation

## Review: Understanding How Data are Absent
    
* Missing Completely at Random (MCAR)
    
* Missing at Random (MAR)
    
* Not Missing at Random (NMAR)

Describe them to your neightbor as if they are not experienced working with data.

## Review MCAR/MAR/NMAR

* **MCAR**: Data is *Missing Completely at Random* if there is no relationship between the missingness of the data and any values, observed or missing.

* **MAR**:  Data is *Missing at Random* if there is a systematic relationship between the propensity of missing values and the observed data, but not the missing data. 

* **NMAR**: Data is *Not Missing at Random* if there is a relationship between the propensity of a value to be missing and its values. 

### Discussion Question

For each of the following datasets, decide whether they are MCAR, MAR, NMAR:
* Self-reported income in the census.
* Measurements transmitted back from the Hubble Space Telescope.
* The Tail Number of a plane in the delayed flights data. 
* SAT scores reported by an institution for College Ranking scores.

## MCAR definition:

Suppose we have:
- a dataset $Y$ with observed values $Y_{obs}$ and missing values $Y_{mis}$.
- a parameter $\psi$ independent of the dataset.

**MCAR**: Data is *Missing Completely at Random* if 

$$Pr({\rm data\ is\ present\ } | Y_{obs}, Y_{mis}, \psi) = Pr({\rm data\ is\ present\ } |\ \psi)$$

That is, adding information on the dataset doesn't change likelihood data is missing!

## MAR definition

Suppose we have:
- a dataset $Y$ with observed values $Y_{obs}$ and missing values $Y_{mis}$.
- a parameter $\psi$ independent of the dataset.


**MAR**: Data is *Missing at Random* if 

$$Pr({\rm data\ is\ present\ } | Y_{obs}, Y_{mis}, \psi) = Pr({\rm data\ is\ present\ } |\ Y_{obs}, \psi)$$

That is, *MAR data is actually MCAR, conditional on $Y_{obs}$*

## NMAR definitions using distributions

Suppose we have:
- a dataset $Y$ with observed values $Y_{obs}$ and missing values $Y_{mis}$.
- a parameter $\psi$ independent of the dataset.


**NMAR**: Data is *Not Missing at Random* if 

$$Pr({\rm data\ is\ present\ }| Y_{obs}, Y_{mis}, \psi)$$

does not simplify. That is, in $NMAR$ data, missingness is dependent on the missing value itself.

## How to assess the mechanism of missingness: NMAR
* Cannot determine NMAR from the data alone; it depends on the unobserved.
* Must be reasoned by the data generating process, or more data should be collected.
* The strength of the dependence on $Y_{mis}$ influences the strength of MNAR


### Discussion Question

* Consider a dataset of survey data of people's self-reported weights.
    - The data contain an identifier and weight; nothing else.
* Is the data likely NMAR? Why?
* What data might you collect to make it not NMAR?

## How to assess the mechanism of missingness: MAR

* Data are MAR if missingness only depends on *obsvered* data.
* Data is MAR if it's determined to not be NMAR (assumption on data generating process).
* Adding further measurements may reduce the effect of NMAR.
    - income in census is MNAR; less so when adding geography, education, race...

## How to assess the mechanism of missingness: MCAR

Given the assumption of MAR, you can test if a data are MCAR.

A column `c_test` is MCAR if its missingness $R$ is independent of the data.
* For each column `c`, check that the missingness rates of `c_test` are the same across values of `c`.
* That is, the distribution of `c` when `c_test.isull()` is 'the same' as the distribution of `c` when `c_test.notnull()`.
* The phrase 'the same' needs to be made statistically precise!

### Checking data are MCAR: heights data
* Start with complete dataset of child/parent heights.
* Blank out rows to create MCAR data.

In [ ]:
heights = pd.read_csv('galton.csv')

heights['child'] = heights.childHeight
heights = heights.drop(['family', 'midparentHeight', 'children', 'childNum', 'childHeight'], axis=1)
heights.head()

In [ ]:
# distribution of heights
pd.plotting.scatter_matrix(heights.drop('gender', axis=1));

In [ ]:
# create missing data

heights_mcar = heights.copy()
idx = heights_mcar.sample(frac=0.3).index
heights_mcar.loc[idx, 'child'] = np.NaN

In [ ]:
heights_mcar.isnull().mean()

### Verifying that child heights are MCAR in `heights_mcar`
* Check the data look the 'same' when `height` is null vs not-null
    - Is the empirical distribution of gender similar for null/not-null?
    - Is the empirical distribution of heights similar for null/not-null?

In [ ]:
# Gender
(
    heights_mcar
    .assign(is_null=heights_mcar.child.isnull())
    .groupby(['is_null', 'gender'])
    .size()
    .rename('cnt')
    .reset_index()
    .pivot('is_null', 'gender', 'cnt')
    .apply(lambda x:x/x.sum(), axis=0)
).T.plot(kind='bar');

In [ ]:
# heights: counts
(
    heights_mcar
    .assign(is_null=heights_mcar.child.isnull())
    .groupby('is_null')
    .father
    .plot(kind='hist', legend=True, title='father height by missingness of child height')
);

In [ ]:
# heights: distributions
(
    heights_mcar
    .assign(is_null=heights_mcar.child.isnull())
    .groupby('is_null')
    .father
    .plot(kind='kde', legend=True, title='father height by missingness of child height')
);

### Child heights data: MAR
* MAR is an *assumption* from the data
* Once MAR is assumed, can show data is *not* MCAR

In [ ]:
# build MAR dataset

heights_mar = heights.copy()
for i, row in heights.iterrows():
    rand = np.random.uniform()
    if (row['father'] > 72) and rand < 0.5:
        heights_mar.loc[i, 'child'] = np.NaN
    elif (row['gender'] == 'female') and rand > 0.7:
        heights_mar.loc[i, 'child'] = np.NaN


In [ ]:
# different missingness rates -- not MCAR

(
    heights_mar
    .assign(is_null=heights_mar.child.isnull())
    .groupby(['is_null', 'gender'])
    .size()
    .rename('cnt')
    .reset_index()
    .pivot('gender', 'is_null', 'cnt')
    .apply(lambda x:x/x.sum())
).plot(kind='bar');

In [ ]:
# Different missingness rates -- not MCAR
# Why is the right side of the graph different?
(
    heights_mar
    .assign(is_null=heights_mar.child.isnull())
    .groupby('is_null')
    .father
    .plot(kind='kde', legend=True, title='father height by missingness of child height')
);

## When are two distributions 'the same'?
* Need a way to measure the 'distance' between distributions (i.e. a statistic)
* Need a way to determine when a large distance is meaningful (i.e. inference)
    - discussed later in the course!

## Measuring the difference between distributions

* Total Variation Distance measures the distance between distributions:
    - see https://www.inferentialthinking.com/chapters/11/2/Multiple_Categories
* If the distribution of a variable when `c` is missing is 'far' from the distribution when `c` is present, then the column is likely *not* MCAR

### Total Variation Distance (TVD)

* Given two distributions $X$ and $Y$, the *total variation distance* is the sum of the absolute difference between the terms of $X$ and the terms in $Y$:

$${\rm TVD} = \sum_i \frac{|x_i - y_i|}{2}$$

* In code:

```
def total_variation_distance(distr_1, distr_2):
    return np.sum(np.abs(distr_1 - distr_2)) / 2
```



In [ ]:
# What is the TVD between the empirical distributions of gender in null/not-null heights data?

# note: same as calculation above!
distributions = (
    heights_mar
    .assign(is_null=heights_mar.child.isnull())
    .pivot_table(columns='is_null', index='gender', values='child', aggfunc='size')
    .apply(lambda x:x/x.sum(), axis=0)
)

distributions

In [ ]:
distributions.plot(kind='bar');

In [ ]:
diff = distributions[True] - distributions[False]
diff

In [ ]:
# why the divide by 2?
# is this value large? what's the context?
np.sum(np.abs(diff))/2

# What to do with missing data?

## What to do with missing data?

* Given a dataset $Y$ with observed values $Y_{obs}$ and missing values $Y_{mis}$.
* $Y$ may appear quite different than $Y_{obs}$:
    - The mean and percentile may be different.
    - The variance may be different.
    - Correlations between variables may be different.

## Simple ways of dealing with missing data

If the data are ______, `.dropna()` doesn't significantly change the data. That is, it produces unbiased estimates of means and variances.

* MCAR (missing completely at random)

* MAR (missing at random)

* NMAR (not missing at random)

* None of the above

## Ways of dealing with missing data: deletion

Using *listwise deletion* (dropping observations) to remove missing values:
* Only works with *MCAR* data. (rare)
* Removes observations with non-null data in other fields. (bad)
* Doesn't further bias the observed dataset.

Explain to yourself why these are true! (Recall DSC 10)

## Ways of dealing with missing data: imputation

Imputation is the act of filling in missing data with plausable values.

* Should be quick and easy to do
* Shouldn't "change" the dataset

These are hard to satisfy!

## Imputation

* imputation with a single value: mean, median, mode
* imputation from a single value, with a model: regression, kNN
* imputation by drawing from a distribution

Each has upsides and downsides; each works differently with different types of missingness.

## Mean imputation

* Imputing a missing value with the mean:
    - preserves the mean of the observed data
    - decreases the variance
    - biases the means across groups when not *MCAR*

## Mean imputation in the `heights` data
* Mean imputation of MCAR data
    - unbiased estimator of the mean.
    - decreases variance.

In [ ]:
# counts of child heights
plt.hist([heights_mcar.child.dropna(), heights.child])
plt.legend(['missing (mcar)', 'full data']);

In [ ]:
# impute with mean
heights_mcar_mfilled = heights_mcar.fillna(heights_mcar.child.mean())

In [ ]:
print(
    'mean (original): %f' % heights.child.mean(),
    'mean (missing):  %f' % heights_mcar.child.mean(),
    'mean (mean imp): %f' % heights_mcar_mfilled.child.mean(),
    sep='\n'
)

In [ ]:
# Why is this smaller, given formula for std
print(
    'std (original): %f' % heights.child.std(),
    'std (missing):  %f' % heights_mcar.child.std(),
    'std (mean imp): %f' % heights_mcar_mfilled.child.std(),
    sep='\n'
)

In [ ]:
# counts of child heights
plt.hist([heights.child, heights_mcar.child.dropna(), heights_mcar_mfilled.child])
plt.legend([ 'full data', 'missing (mcar)', 'imputed']);

## Mean imputation and MAR data

* Mean imputation leads to biased estimates of mean across groups, when using MAR data.
* Since MAR is MCAR within each group, can do group-wise mean imputation.

In [ ]:
heights_mar1 = heights.copy()

# blank out data for males at higher rate
weights = heights_mar1.gender.replace({'male': 0.95, 'female': 0.05})
idx = heights_mar1.sample(frac=0.3, weights=weights).index
heights_mar1.loc[idx, 'child'] = np.NaN

In [ ]:
# The observed vs true distribution
plt.hist([heights.child, heights_mar1.child]);
plt.legend([ 'full data','missing (mar)']);

In [ ]:
# naive mean imputation
heights_mar1_mfilled = heights_mar1.fillna(heights_mar1.child.mean())

In [ ]:
print(
    'mean (original): %f' % heights.child.mean(),
    'mean (missing):  %f' % heights_mar1.child.mean(),
    'mean (mean imp): %f' % heights_mar1_mfilled.child.mean(),
    sep='\n'
)

In [ ]:
print(
    'std (original): %f' % heights.child.std(),
    'std (missing):  %f' % heights_mar1.child.std(),
    'std (mean imp): %f' % heights_mar1_mfilled.child.std(),
    sep='\n'
)

In [ ]:
plt.hist([heights.child, heights_mar1.child, heights_mar1_mfilled.child]);
plt.legend([ 'full data','missing (mar)', 'imputed']);

In [ ]:
# Biased mean by groups!
# How is this connected to simpson's paradox?
pd.concat([
    heights.groupby('gender').child.mean().rename('full'),
    heights_mar1.groupby('gender').child.mean().rename('missing (mar)'),
    heights_mar1_mfilled.groupby('gender').child.mean().rename('imputed')
], axis=1)

In [ ]:
# Impute separately by gender!

## Conclusions: imputation with single values
* Imputing missing data in a column with the mean of the column:
    - faithfully reproduces the mean of the observed dataset,
    - reduces the variance,
    - biases relationships of the column with other columns.
    
* Similar with other statistics (median, mode).

### Discussion Question

* Consider the income reporting in the US Census. 
* Suppose we impute missing salaries with the mean overall income.
* Is there more bias in:
    - (low-paying) service jobs or 
    - (high-paying) executive jobs?
    
Hint: what does the distribution of incomes look like? Where is the mean/median?

## Imputing missing values using distributions
* We can *probalistically* impute missing data from a distribution.
    - Fill in missing data by drawing from the distribution of *non-missing* data.
* Recall: using `.sample`, we can draw from an (empirical) distribution.

### Imputing `heights_mcar` data by drawing from a distribution.

Steps:
* Sample values of child heights from the observed heights.
* Fill in missing child heights by using these draws from the observed heights.

In [ ]:
num_null = heights_mcar.child.isnull().sum() # number of nulls
fill_values = heights_mcar.child.dropna().sample(num_null, replace=True)  # draw fill vals from distribution
fill_values.index = heights_mcar.loc[heights_mcar.child.isnull()].index  # align the index
heights_mcar_dfilled = heights_mcar.fillna({'child': fill_values.to_dict()})  # fill the vals

In [ ]:
print(
    'mean (original):  %f' % heights.child.mean(),
    'mean (missing):   %f' % heights_mcar.child.mean(),
    'mean (distr imp): %f' % heights_mcar_dfilled.child.mean(),
    sep='\n'
)

In [ ]:
print(
    'std (original):  %f' % heights.child.std(),
    'std (missing):   %f' % heights_mcar.child.std(),
    'std (distr imp): %f' % heights_mcar_dfilled.child.std(),
    sep='\n'
)

In [ ]:
plt.hist([heights.child, heights_mcar.child, heights_mcar_dfilled.child], density=True);
plt.legend([ 'full data','missing (mcar)', 'distr imputed']);

## Observations
* We only drew missing values from observed values. What if a missing value was previously unobserved?
    - better to bin the data and draw from bins instead! 
    - use `np.histogram` to bin data.

* How does this generalize to categoricals?

## Observations

* We created an imputed dataset using randomness to preserve variance
* Our imputation could have been different, had we run it again
* *Multiple imputation*: generate multiple imputed datasets and aggregate each result!
    - Should remind you of bootstrap estimation.

# Multiple Imputation
(Donald Rubin)

Multiple imputation is a 3 step process:

* **Imputation**: Impute a missing data multiple times (m times)
* **Analysis**: Analyze each complete dataset separetly (m sets)
* **Pooling**: Combine multiple analysis result. M estimates result in the final estimate.

![](./imgs/mult_imp.png)



### Multiple Imputation: steps

0. Start with observed and incomplete data. 
1. Create several **imputed** versions of the data by drawing from a distribution of plausible values.
    - The imputed datasets are identical for the observed data entries, 
    - they differ in the imputed values. 
    - The differences reflect our **uncertainty** about what value to impute.

2. Then we estimate the parameters of interest from **each** imputed dataset.
3. The last step is to pool the m parameter estimates into one estimate and to estimate its variance.

In [ ]:
# heights_mcar is our incomplete data

def create_imputed(col):
    num_null = col.isnull().sum()
    fill_values = col.dropna().sample(num_null, replace=True)
    fill_values.index = col.loc[col.isnull()].index
    return col.fillna(fill_values.to_dict())


In [ ]:
create_imputed(heights_mcar.child).head()

In [ ]:
mult_imp = pd.concat([create_imputed(heights_mcar.child).rename(k) for k in range(100)], axis=1)
mult_imp.head()

In [ ]:
# plot 15 random imputations
mult_imp.sample(15, axis=1).plot(kind='kde', alpha=0.5, legend=False);

In [ ]:
# sampling distribution of imputed means
mult_imp.mean().plot(kind='hist', bins=20);

In [ ]:
# doesn't decrease the standard deviations of imputed data
mult_imp.std().plot(kind='hist', bins=50)
std_incomplete = heights_mcar.child.std()
plt.plot([std_incomplete, std_incomplete], [0,8]);

### Discussion Question

* Suppose we *average* all the multiple imputed heights and impute data using that average.
* Does the variance of the imputed data:
    - Remain the same?
    - Decrease?

In [ ]:
imputed = mult_imp.mean(axis=1)

In [ ]:
print(
    'std (original):  %f' % heights.child.std(),
    'std (missing):   %f' % heights_mcar.child.std(),
    'std (distr imp): %f' % imputed.std(),
    sep='\n'
)

In [ ]:
plt.hist([heights.child, heights_mcar.child, imputed], density=True);
plt.legend([ 'full data','missing (mcar)', 'distr imputed']);

## Missingness, conditional on multiple variables
* Use multiple imputation
* Imputate from a distribution for each variable
* Aggregate the multiple imputations